## 🤔 Config

In [ ]:
#|default_exp utils.config

In [ ]:
# |hide
from nbdev.showdoc import *

In [ ]:
# |hide
# |export
from copy import copy
from types import SimpleNamespace
from typing import Callable, Union, Optional, TypeVar
from contextlib import contextmanager

import numpy as np

In [ ]:
# |exporti

class Config(SimpleNamespace):
    "Config"
    def __init__(self,
            precision     = 3,    # Digits after `.`
            threshold_max = 3,    # .abs() larger than 1e3 -> Sci mode
            threshold_min = -4,   # .abs() smaller that 1e-4 -> Sci mode
            sci_mode      = None, # Sci mode (2.3e4). None=auto
            indent        = 2,    # Indent for .deeper()
            color         = True, # ANSI colors in text
            repr          = None, # Use func e.g. `lovely` for `repr(np.ndarray)`
            str           = None, # Use func e.g. `lovely` for `str(np.ndarray)`
            plt_seed      = 42    # Sampling seed for `plot`. None=random.  
    ):
        super().__init__(**{k:v for k,v in locals().items() if k not in ["self", "__class__"]})

_defaults = Config()

_config = copy(_defaults)

### Defaults:

In [ ]:
# |echo: false
DocmentTbl(Config)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| precision | int | 3 | Digits after `.` |
| threshold_max | int | 3 | .abs() larger than 1e3 -> Sci mode |
| threshold_min | int | -4 | .abs() smaller that 1e-4 -> Sci mode |
| sci_mode | NoneType | None | Sci mode (2.3e4). None=auto |
| indent | int | 2 | Indent for .deeper() |
| color | bool | True | ANSI colors in text |
| repr | NoneType | None | Use func e.g. `lovely` for `repr(np.ndarray)` |
| str | NoneType | None | Use func e.g. `lovely` for `str(np.ndarray)` |
| plt_seed | int | 42 | Sampling seed for `plot`. None=random. |

In [ ]:
# |exporti

# Allows passing None as an argument to reset the 
class _Default():
    def __repr__(self):
        return "Ignore"
D = _Default()
Default = TypeVar("Default")

In [ ]:
# |export
def set_config( precision       :Optional[Union[Default,int]]     =D,
                threshold_min   :Optional[Union[Default,int]]     =D,
                threshold_max   :Optional[Union[Default,int]]     =D,
                sci_mode        :Optional[Union[Default,bool]]    =D,
                indent          :Optional[Union[Default,bool]]    =D,
                color           :Optional[Union[Default,bool]]    =D,
                repr            :Optional[Union[Default,Callable]]=D,
                str             :Optional[Union[Default,Callable]]=D,
                plt_seed        :Optional[Union[Default,int]]     =D,
                ) -> None:

    "Set config variables"
    args = locals().copy()
    for k,v in args.items():
        if v != D:
            
            # set_config(repr=func)             -> Set both `repr` and `str`.
            # set_config(repr=func, str=None)   -> Set `repr`, unset `str``
            # set_config(str=func)              -> Set `str`` only, don't tuch `repr``
            # set_config(repr=None)             -> Unset `repr`and `str``
            # set_config(str=None)              -> Unset `str` only

            if k == "repr":
                np.set_string_function(v, True)
                if args["str"] == D:
                    np.set_string_function(v, False)
            if k == "str":
                np.set_string_function(v, False)

            if v is None:
                setattr(_config, k, getattr(_defaults, k))
            else:
                setattr(_config, k, v)

In [ ]:
# |export
def get_config():
    "Get a copy of config variables"
    return copy(_config)

In [ ]:
# |export
@contextmanager
def config( precision       :Optional[Union[Default,int]]     =D,
            threshold_min   :Optional[Union[Default,int]]     =D,
            threshold_max   :Optional[Union[Default,int]]     =D,
            sci_mode        :Optional[Union[Default,bool]]    =D,
            indent          :Optional[Union[Default,bool]]    =D,
            color           :Optional[Union[Default,bool]]    =D,
            repr            :Optional[Union[Default,Callable]]=D,
            str             :Optional[Union[Default,Callable]]=D,
            plt_seed        :Optional[Union[Default,int]]     =D,
            ):
    "Context manager for temporarily setting config options"
    new_opts = { k:v for k, v in locals().items() if v != D}
    old_opts = copy(get_config().__dict__)


    try:
        set_config(**new_opts)
        yield
    finally:
        set_config(**old_opts)

## Examples

In [ ]:
# |hide

# In the next cell I'm importing the functios from the exported .py as part of the examples code
# This overrides the functions defined above!
# Make sure the code is in sync

import nbdev; nbdev.nbdev_export()

In [ ]:
from lovely_numpy import Lo, lovely, set_config, get_config, config

### Precision

In [ ]:
set_config(precision=5)
Lo(np.array([1., 2, 3]))

array[3] x∈[1.00000, 3.00000] μ=2.00000 σ=0.81650 [1.00000, 2.00000, 3.00000]

### Scientific mode

In [ ]:
set_config(sci_mode=True) # Force always on
Lo(np.array([1., 2, 3]))

array[3] x∈[1.00000e+00, 3.00000e+00] μ=2.00000e+00 σ=8.16497e-01 [1.00000e+00, 2.00000e+00, 3.00000e+00]

### Color on/off

In [ ]:
set_config(color=False) # Force always off
Lo(np.array(np.nan))

array NaN! nan

### Repr and str

In [ ]:
set_config(repr=lovely)
np.array([1,2])

array[2] i64 μ=1.50000e+00 σ=5.00000e-01 [1, 2]

:::{.callout-note}
I expect that most people would want `repr` and `str` to do the same thing in this context.\
Setting `repr` also sets `str`. Pass `str=None` to only set `repr`
:::

```py
set_config(repr=func)             # Set both `repr` and `str`.
set_config(repr=func, str=None)   # Set `repr`, unset `str`
set_config(str=func)              # Set `str` only, don't touch `repr`
set_config(repr=None)             # Unset `repr`and `str`
set_config(str=None)              # Unset `str` only
```

### Reser to defaults

In [ ]:
set_config(precision=None, sci_mode=None, color=None, repr=None)

In [ ]:
Lo(np.array([1., 2, np.nan]))

array[3] μ=1.500 σ=0.500 NaN! [1.000, 2.000, nan]

In [ ]:
np.array([1,2])

array([1, 2])

### Context manager

In [ ]:
with config(sci_mode=True):    
    print(Lo(np.array([1., 2, 3])))

array[3] x∈[1.000e+00, 3.000e+00] μ=2.000e+00 σ=8.165e-01 [1.000e+00, 2.000e+00, 3.000e+00]


In [ ]:
Lo(np.array([1., 2, 3]))

array[3] x∈[1.000, 3.000] μ=2.000 σ=0.816 [1.000, 2.000, 3.000]

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()